In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "Serrina123"
port = "32222"
database = "AAC"
collection = "animals"
shelter = AnimalShelter(username, password)

#This will create the dataframe
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('ProjectTwo')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Serrina Paasch SNHU CS-340 Dashboard'))),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=True,
        row_selectable="single", #This will allow for a row to be selected
        selected_rows=[],
        filter_action="native", #This will allow for a filter to be selected
        sort_action="native", #This will allow for sorting
        page_action="native", #This will enable pagination
        page_current=0, #This will set start page
        page_size=10, #This will set rows per page

    ),
    html.Br(),
     html.Hr(),
     html.Div( #Div for the map
            id='map-id',
            className='col s12 m6',
            )
    ])

#############################################
# Interaction Between Components / Controller
#############################################
#This will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('map-id', "children"),    
    [Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(virtualRows):
    
    print(virtualRows)

    #Austin Texas is [30.75, -97.48]
    
    #This will create the view
    if not virtualRows: #This will build a default view if there are no selected lines
        markerArray = (30.75,-97.48) #This is the default marker at Austin Animal Shelter
        toolTip = "Austin Animal Center"
        popUpHeading = "Austin Animal Center"
        popUpParagraph = "Shelter Home Location"
        
    else: #This will build the contextual views based on the selection
        dff = pd.DataFrame(df.iloc[virtualRows]) #This converts the datatable to a dataframe
        coordLat = float(dff['location_lat'].to_string().split()[1]) #This strips out the lat
        coordLong = float(dff['location_long'].to_string().split()[1]) #This strips out the long
        markerArray = (coordLat, coordLong) #This will build the array based on selection
        
        toolTip = dff['breed']
        popUpHeading = "Animal Name"
        popUpParagraph = dff['name']

    return [dl.Map(style={'width': '600px', 'height': '300px'}, center=[30.75,-97.48],
                   zoom=10, children=[dl.TileLayer(id="base-layer-id"),
                                      dl.Marker(position=markerArray, children=[
                                          dl.Tooltip(toolTip),
                                          dl.Popup([
                                              html.H1(popUpHeading),
                                              html.P(popUpParagraph)
                                          ])
                                      ])
                                     ])
           ]
    
app.run_server(debug=True)

Hello Animal Shelter
Working in Read Method...
Dash app running on http://127.0.0.1:8508/
None
[]
[1]
[2]
[3]
[7]
[9]
[8]
[1]
